<a href="https://colab.research.google.com/github/divyanshsaxena21/Bureau_Assignment/blob/main/Bureau_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

In [52]:
# Load data
train_data = pd.read_csv('Assignment_Train.csv')
test_data = pd.read_csv('Assignment_Test.csv')

In [53]:
train_data.head(1)

,DEALER ID,APPLICATION LOGIN DATE,HDB BRANCH NAME,HDB BRANCH STATE,FIRST NAME,MIDDLE NAME,LAST NAME,mobile,AADHAR VERIFIED,Cibil Score,...,Phone Social Premium.shaadi,Phone Social Premium.skype,Phone Social Premium.toi,Phone Social Premium.whatsapp,Phone Social Premium.yatra,Phone Social Premium.zoho,phone_digitalage,phone_nameMatchScore,phone_phoneFootprintStrengthOverall,Application Status
0,106989,07/20/2022,DELHI-SF,DELHI,SUNIL,NaN,CHANDER,9210574080,NO,726,...,0.0,0.0,1.0,NaN,NaN,0.0,5324.0,67.222222,High,APPROVED


In [54]:
# Convert 'APPLICATION LOGIN DATE' to datetime
train_data['APPLICATION LOGIN DATE'] = pd.to_datetime(train_data['APPLICATION LOGIN DATE'], format='%m/%d/%y', errors='coerce')

# Extract features from the 'APPLICATION LOGIN DATE'
train_data['Month'] = train_data['APPLICATION LOGIN DATE'].dt.month
train_data['Day'] = train_data['APPLICATION LOGIN DATE'].dt.day
train_data['Year'] = train_data['APPLICATION LOGIN DATE'].dt.year

# Drop the original 'APPLICATION LOGIN DATE' column
train_data = train_data.drop('APPLICATION LOGIN DATE', axis=1)

# Save the updated DataFrame to a new CSV file
train_data.to_csv('Assignment_Train_Updated.csv', index=False)

In [55]:
# Extract features and target variable
X = train_data.drop(['Application Status', 'DEALER ID'], axis=1)
Y = train_data['Application Status']


In [56]:
# Convert categorical variables to numerical
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

In [57]:
# Handle missing values
imputer = SimpleImputer(strategy='most_frequent')

# Exclude the datetime column from imputation
X_no_date = X.select_dtypes(exclude=['datetime64'])
X_no_date = imputer.fit_transform(X_no_date)

# Convert the result back to DataFrame and re-insert the date column
X = pd.DataFrame(X_no_date, columns=X.select_dtypes(exclude=['datetime64']).columns)

In [58]:
# Standardize features
scaler = StandardScaler()

# Exclude the datetime column from scaling
X_no_date = X.select_dtypes(exclude=['datetime64'])
X_no_date = scaler.fit_transform(X_no_date)

# Convert the result back to DataFrame and re-insert the date column
X = pd.DataFrame(X_no_date, columns=X.select_dtypes(exclude=['datetime64']).columns)

In [59]:
# Split data for training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [60]:
X_train.shape

(8000, 55)

In [61]:
model = RandomForestClassifier(random_state=2)
model.fit(X_train, Y_train)

RandomForestClassifier(random_state=2)

In [62]:
# Evaluate model
Y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('Classification Report:\n', classification_report(Y_test, Y_pred))

Accuracy: 0.8945
Classification Report:
               precision    recall  f1-score   support

    APPROVED       0.92      0.92      0.92      1305
    DECLINED       0.85      0.84      0.85       695

    accuracy                           0.89      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.89      0.89      0.89      2000



In [63]:
# Prepare submission
# Process the test data similarly
X_test = test_data.drop('UID', axis=1)

# Convert 'APPLICATION LOGIN DATE' to datetime
X_test['APPLICATION LOGIN DATE'] = pd.to_datetime(X_test['APPLICATION LOGIN DATE'])
# Extract date features
X_test['Day'] = X_test['APPLICATION LOGIN DATE'].dt.day
X_test['Month'] = X_test['APPLICATION LOGIN DATE'].dt.month
X_test['Year'] = X_test['APPLICATION LOGIN DATE'].dt.year
# Drop the original date column
X_test = X_test.drop('APPLICATION LOGIN DATE', axis=1)

for col in X_test.select_dtypes(include=['object']).columns:
    # Check if the column exists in X_train before proceeding
    if col in X_train.columns:
        # Create a list of unique values seen in training and testing data
        unique_vals = list(set(X_train[col].astype(str).unique()).union(set(X_test[col].astype(str).unique())))
        # Fit the label encoder on these unique values
        label_encoder.fit(unique_vals)
        X_test[col] = label_encoder.transform(X_test[col].astype(str))

# Ensure the columns in X_test are in the same order as X_train
X_test = X_test[X_train.columns]

X_test = imputer.transform(X_test)
X_test = scaler.transform(X_test)

<ipython-input-63-a2e6cfbe901d>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X_test['APPLICATION LOGIN DATE'] = pd.to_datetime(X_test['APPLICATION LOGIN DATE'])
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [64]:
# Predict on test data
test_predictions = model.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({
    'UID': test_data['UID'],
    'Application Status': test_predictions
})

# Save submission
submission.to_csv('predictions.csv', index=False)

print("Submission file created.")

Submission file created.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
